In [1]:
# Load feature vectors:

import cPickle

with open('/Users/sunyambagga/Desktop/combined_feature_vector3_cbow.pkl', 'rb') as f:
    inp = cPickle.load(f)
print "Feature vectors: ", len(inp)

avg = 0
for l in inp.values():
    avg += l['competitor_num']
    
print "Average number of competitor answers: ", avg/len(inp)

fv = {}

ORDER = ['url_tag', 'pic', 'code', 'ans_len', 'readability1', 'readability2', 'QA_sim', 'timeSlot', 'ave_ans_sim', 
         'min_ans_sim', 'max_ans_sim', 'competitor_num', 'ans_index', 'owner_scoreA', 'owner_scoreQ']

for ansID in inp.keys():
    temp = []
    for feature_name in ORDER:
        temp.append(inp[ansID][feature_name])
    
    fv[ansID] = temp
print len(fv.keys())

import pandas as pd

# Load only relevant labels:
path = '/Users/sunyambagga/Desktop/labeled_answerIDs.csv'
blah = pd.read_csv(path)

answer_ids_available = inp.keys()

labels = blah.loc[blah['Answer_ID'].isin(answer_ids_available)]
print labels.nunique()
labels.head()

Feature vectors:  411789
Average number of competitor answers:  3.62847963399
411789
Answer_ID    411789
Label             2
dtype: int64


,Answer_ID,Label
3,9437192,BestAnswer
4,9437193,BestAnswer
9,19922963,BestAnswer
10,14854830,BestAnswer
14,3145756,BestAnswer


In [2]:
lab = labels.set_index('Answer_ID').T.to_dict('records')
LABELS = lab[0]
LABELS

{3670017: 'Non_BestAnswer',
 9437192: 'BestAnswer',
 9437193: 'BestAnswer',
 30408714: 'BestAnswer',
 9437202: 'Non_BestAnswer',
 19922963: 'BestAnswer',
 5417646: 'Non_BestAnswer',
 3145756: 'BestAnswer',
 26214440: 'BestAnswer',
 2097197: 'Non_BestAnswer',
 3142456: 'Non_BestAnswer',
 27263025: 'Non_BestAnswer',
 5242930: 'Non_BestAnswer',
 30408755: 'Non_BestAnswer',
 13631541: 'BestAnswer',
 31457335: 'Non_BestAnswer',
 35651642: 'BestAnswer',
 3145790: 'Non_BestAnswer',
 28311619: 'BestAnswer',
 14680133: 'BestAnswer',
 28311625: 'BestAnswer',
 30408779: 'Non_BestAnswer',
 3145804: 'Non_BestAnswer',
 36700237: 'BestAnswer',
 11534415: 'BestAnswer',
 24117328: 'BestAnswer',
 15728722: 'BestAnswer',
 3145811: 'BestAnswer',
 35651671: 'Non_BestAnswer',
 4194392: 'Non_BestAnswer',
 9437273: 'Non_BestAnswer',
 11534427: 'Non_BestAnswer',
 14680161: 'Non_BestAnswer',
 4194402: 'BestAnswer',
 27263075: 'Non_BestAnswer',
 14680165: 'Non_BestAnswer',
 35651686: 'Non_BestAnswer',
 1048685: 

In [3]:
import numpy as np

X = []
Y = []

for answer_id in fv.keys():
    X.append(fv[answer_id])
    if LABELS[answer_id] == 'BestAnswer':
        Y.append(1)
    elif LABELS[answer_id] == 'Non_BestAnswer':
        Y.append(0)
    else:
        print "WTF"

print "Total X: ", len(X)
print "Total Y: ", len(Y)

# Train test split:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print "Train: ", len(X_train)
print len(Y_train)
print "Test: ", len(X_test)
print len(Y_test)

Total X:  411789
Total Y:  411789
Train:  329431
329431
Test:  82358
82358


In [4]:
ba_train = 0
ba_test = 0

for i in Y_train:
    if i == 1:
        ba_train += 1
        
for i in Y_test:
    if i == 1:
        ba_test += 1

print "Best answers in train: ", ba_train
print "Best answers in test: ", ba_test

Best answers in train:  111932
Best answers in test:  27728


In [5]:
# Make classes balanced:
# Undersampling using Provost (year 2000)

finalX_train = []
finalY_train = []

# finalX_test = []
# finalY_test = []

np.random.seed(7)

c_tr = 0
c_test = 0

for (x,y) in zip(X_train, Y_train):
    if y == 1:
        finalX_train.append(x)
        finalY_train.append(y)
        
    elif y == 0:
        # Stop at ba_train:
        if c_tr >= ba_train:
            continue
            
        if np.random.random() < 0.8:
            finalX_train.append(x)
            finalY_train.append(y)
            c_tr += 1
            


# for (x,y) in zip(X_test, Y_test):
#     if y == 1:
#         finalX_test.append(x)
#         finalY_test.append(y)
#     elif y == 0:
#         # Stop at ba_test:
#         if c_test >= ba_test:
#             continue
            
#         if np.random.random() < 0.8:
#             finalX_test.append(x)
#             finalY_test.append(y)
#             c_test += 1


print len(finalX_train)
print len(finalY_train)

# print len(finalX_test)
# print len(finalY_test)

print "Total dataset: ", len(finalX_train)+len(X_test)

223864
223864
Total dataset:  306222


In [6]:
# Making sure it's balanced:
c = 0
for i in finalY_train:
    if i == 0:
        c += 1
print c

111932


In [8]:
X = finalX_train + X_test
Y = finalY_train + Y_test
print len(X)
print len(Y)

306222
306222


In [15]:
co = 0
for i in Y:
    if i == 0:
        co += 1
print co
# print co*2

166562


In [16]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1400, num = 7)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print random_grid

{'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]}


In [17]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 

rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=20, cv=3, verbose=5, scoring='f1', random_state=42, n_jobs = -1)

rf_random.fit(X, Y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1400, min_samples_split=10, max_features=sqrt, max_depth=70 
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1400, min_samples_split=10, max_features=sqrt, max_depth=70 
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1400, min_samples_split=10, max_features=sqrt, max_depth=70 
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1400, min_samples_split=5, max_features=auto, max_depth=None 
[CV]  bootstrap=True, min_samples_leaf=2, n_estimators=1400, min_samples_split=10, max_features=sqrt, max_depth=70, score=0.628937661179, total=30.2min
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1400, min_samples_split=5, max_features=auto, max_depth=None 
[CV]  bootstrap=True, min_samples_leaf=2, n_estimators=1400, min_samples_split=10, max_features=sqrt, max_depth=70, score=0.630524697373, total=30.1min
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1400

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 68.9min


[CV] bootstrap=False, min_samples_leaf=1, n_estimators=200, min_samples_split=2, max_features=auto, max_depth=30 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=200, min_samples_split=2, max_features=auto, max_depth=30, score=0.622317409928, total= 5.4min
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=200, min_samples_split=2, max_features=auto, max_depth=30 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=200, min_samples_split=2, max_features=auto, max_depth=30, score=0.623145272135, total=12.4min
[CV] bootstrap=False, min_samples_leaf=2, n_estimators=1400, min_samples_split=2, max_features=auto, max_depth=70 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=200, min_samples_split=2, max_features=auto, max_depth=30, score=0.62243677561, total=12.7min
[CV] bootstrap=False, min_samples_leaf=2, n_estimators=1400, min_samples_split=2, max_features=auto, max_depth=70 
[CV]  bootstrap=True, min_samples_leaf=2, n_estimators=800, min_samples_split=2, max_featu

[CV] bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=10, max_features=sqrt, max_depth=80 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=10, max_features=sqrt, max_depth=80, score=0.626393728223, total=36.1min
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=800, min_samples_split=2, max_features=auto, max_depth=20 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=10, max_features=sqrt, max_depth=80, score=0.629487332114, total=35.9min
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=1400, min_samples_split=5, max_features=auto, max_depth=70, score=0.627131917856, total=36.1min
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=800, min_samples_split=2, max_features=auto, max_depth=20 
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=800, min_samples_split=2, max_features=auto, max_depth=20 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=10, m

[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 353.6min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='f1', verbose=5)

In [18]:
rf_random.best_score_

0.6319890416745993

In [19]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}